# Import Packages

In [1]:
import keras
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout
from keras.utils import to_categorical
import math
import imageio
from pathlib import Path
from ann_visualizer.visualize import ann_viz

Using TensorFlow backend.


# Check If GPU Is Available

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6683341663563678150
]


# Prepare Data From Mnist

In [3]:
(X_Train, Y_Train), (X_Test, Y_Test) = keras.datasets.mnist.load_data()
X_Train = X_Train.reshape(60000,28,28,1)
X_Test = X_Test.reshape(10000,28,28,1)
Y_Train = to_categorical(Y_Train)
Y_Test = to_categorical(Y_Test)

# Build CNN Framework

In [4]:
Model = Sequential()
Model.add(Conv2D(64, kernel_size=3, activation="relu", input_shape=(28,28,1)))
Model.add(Dropout(0.1))
Model.
Model.add(Conv2D(32, kernel_size=3, activation="relu"))
Model.add(Dropout(0.1))
Model.add(Flatten())
Model.add(Dense(10, activation="softmax"))
Model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        18464     
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)      

# Create CNN Map

In [5]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Users/Harib/.conda/envs/tensorflow/Library/bin/graphviz/'

ann_viz(Model, view=False, filename="HandwritingDigitsRecognition.gv", title="Handwriting Digits Recognition")

# Compile & Train CNN

In [6]:
Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
Model.fit(X_Train, Y_Train, validation_data=(X_Test, Y_Test), epochs=5)

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 187s 3ms/step - loss: 7.4945 - acc: 0.5326 - val_loss: 5.7998 - val_acc: 0.6381
Epoch 2/5
60000/60000 [==============================] - 213s 4ms/step - loss: 5.8831 - acc: 0.6342 - val_loss: 5.5564 - val_acc: 0.6549
Epoch 3/5
35712/60000 [================>.............] - ETA: 1:39 - loss: 6.1472 - acc: 0.6182

KeyboardInterrupt: 

# Test A Predictions From The Testing Set

In [ ]:
i = 0
print("Was The AI Right?", np.argmax(Y_Test[[i]]) == np.argmax(Model.predict(X_Test[[i]])))
plt.axis("off")
plt.imshow(cv2.cvtColor(X_Test[i], cv2.COLOR_BGR2RGB))
plt.show()
Model.predict(X_Test[[i]])

# Test All Predictions From The Testing Set

In [ ]:
TestPredictions = Model.predict(X_Test)

ResultList = []
for i, j in enumerate(TestPredictions):
    ResultList.append(np.argmax(TestPredictions[i])==np.argmax(Y_Test[i]))
print("Testing Set Accuracy - ", np.mean(ResultList))

# Test A Drawing I've Made

In [ ]:
i=5

base_path = Path("__file__").parent
file_path = str((base_path / "../CNN/My Images/").resolve())
temp = str(file_path)+"\\"+str(i)+".png"

img = cv2.imread(temp,0)
img2 = cv2.bitwise_not(img)
img3 = img2.reshape(1,28,28,1)

print("Was The AI Right?", i == np.argmax(Model.predict(img3)))
plt.axis("off")
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.show()
Model.predict(img3)

# Test All The Drawings I've Made

In [ ]:
base_path = Path("__file__").parent
file_path = str((base_path / "../CNN/My Images/").resolve())
temp = str(file_path)+"\\?.png"
print(temp)

In [ ]:
X_ImageList = []
Y_ImageList = []
for name in glob.glob(temp):
    X_ImageList.append(cv2.imread(name,0))
    Y_ImageList.append(name[name.find(".png")-1])
X_ImageList = np.array(X_ImageList)

TestPredictions = []
for image in X_ImageList:
    image = cv2.bitwise_not(image)
    image = image.reshape(1,28,28,1)
    TestPredictions.append(Model.predict(image))

In [ ]:
ResultList = []
for i, j in enumerate(TestPredictions):
    ResultList.append(np.argmax(TestPredictions[i])==int(Y_ImageList[i]))
print("My Handwriting Accuracy - ", np.mean(ResultList))
print("Incorrect Numbers - ", np.array(Y_ImageList)[np.argwhere(np.array(ResultList)==False).ravel()].tolist())